In [104]:
TARGET_LOCATION = 'Plymouth, MA'
TARGET_LOCATION = 'Jamaica Plain, MA'
TARGET_LOCATION = 'Andover, MA'
TARGET_LOCATION = 'Lynn, MA'
TARGET_LOCATION = 'Chelmsford, MA'
TARGET_LOCATION = 'Golden, CO'
TARGET_LOCATION = 'Rockingham County, New Hampshire'

from model_lib import RedfinModel

In [105]:
redfin = RedfinModel(TARGET_LOCATION, {
  'style': ['SINGLE_FAMILY', 'TOWNHOUSE'],
  'beds': [3, 4, 5],
  'full_baths': [2, 3],
})
train_df = redfin.fetch_data('sold')
#
train_df.head(3)

Fetched properties (157): Rockingham County, New Hampshire sold
filtering column: style allowed_values: ['SINGLE_FAMILY', 'TOWNHOUSE']
filtering column: beds allowed_values: [3, 4, 5]
filtering column: full_baths allowed_values: [2, 3]
Filtered data shape: (52, 44) (from (157, 43))


,property_url,mls,mls_id,status,text,style,full_street_line,street,unit,city,...,agent,agent_email,agent_phones,broker,broker_phone,broker_website,nearby_schools,primary_photo,alt_photos,age
1,https://www.realtor.com/realestateandhomes-det...,BSMA,73255127,SOLD,You won't find another one like it... Expansiv...,SINGLE_FAMILY,3 Pine Knoll Rd,3 Pine Knoll Rd,<NA>,Atkinson,...,Brian & Michelle Healy,brian.healy@verani.com,"[{'number': '6038181330', 'type': 'Mobile', 'e...",<NA>,<NA>,<NA>,Timberlane Regional School District,http://ap.rdcpix.com/e3088eb52455f9ec19d0bf2f9...,http://ap.rdcpix.com/e3088eb52455f9ec19d0bf2f9...,31
2,https://www.realtor.com/realestateandhomes-det...,BSMA,73216140,SOLD,"Built in 1780, this antique home offers a uniq...",SINGLE_FAMILY,235 Main St,235 Main St,<NA>,Hampstead,...,Ian Handel,Ianhandelrealtor@gmail.com,"[{'number': '6035602214', 'type': 'Home', 'ext...",<NA>,<NA>,<NA>,"Pinkerton Academy School District, Hampstead S...",http://ap.rdcpix.com/ac38dd8f90844f3af3c615d17...,http://ap.rdcpix.com/ac38dd8f90844f3af3c615d17...,244
4,https://www.realtor.com/realestateandhomes-det...,BSMA,73239286,SOLD,Here is the ONE you have been manifesting! Wel...,SINGLE_FAMILY,9 Spring St,9 Spring St,<NA>,Windham,...,Olivares & Molina Team of Realty One Group Nest,info@theomteam.com,"[{'number': '9783603441', 'type': 'Mobile', 'e...",<NA>,<NA>,<NA>,"Windham School District, Granite State Arts Ac...",http://ap.rdcpix.com/4e3ead1fec50ab34fde472ad8...,http://ap.rdcpix.com/4e3ead1fec50ab34fde472ad8...,33


In [106]:
train_df.columns.values
# one hot encode columns

array(['property_url', 'mls', 'mls_id', 'status', 'text', 'style',
       'full_street_line', 'street', 'unit', 'city', 'state', 'zip_code',
       'beds', 'full_baths', 'half_baths', 'sqft', 'year_built',
       'days_on_mls', 'list_price', 'list_date', 'sold_price',
       'last_sold_date', 'assessed_value', 'estimated_value', 'lot_sqft',
       'price_per_sqft', 'latitude', 'longitude', 'neighborhoods',
       'county', 'fips_code', 'stories', 'hoa_fee', 'parking_garage',
       'agent', 'agent_email', 'agent_phones', 'broker', 'broker_phone',
       'broker_website', 'nearby_schools', 'primary_photo', 'alt_photos',
       'age'], dtype=object)

In [107]:
redfin.train_from_raw(train_df, train_df[RedfinModel.TARGET_COLUMN], show_debug=True)
print(redfin.trained_columns)


Using columns: ['age' 'beds' 'days_on_mls' 'full_baths' 'half_baths' 'hoa_fee' 'lot_sqft'
 'parking_garage' 'sqft' 'stories' 'city=Atkinson' 'city=Candia'
 'city=Danville' 'city=Deerfield' 'city=Derry' 'city=East Kingston'
 'city=Epping' 'city=Hampstead' 'city=Hampton' 'city=Kingston'
 'city=Londonderry' 'city=Plaistow' 'city=Salem' 'city=Sandown'
 'city=Seabrook' 'city=Windham' 'nearby_schools'
 'nearby_schools=Birches Academy for Academic, Hill View Montessori Charter Public (District), Salem School District, Granite State Arts Academy Charter School'
 'nearby_schools=Birches Academy for Academic, Salem School District, Granite State Arts Academy Charter School'
 'nearby_schools=Birches Academy for Academic, Windham School District, Granite State Arts Academy Charter School, Windham Academy Public Charter School'
 'nearby_schools=Candia School District'
 'nearby_schools=Deerfield School District'
 'nearby_schools=Epping School District'
 'nearby_schools=Great Bay Elearning Charter Sc

In [108]:
mean_score = redfin.show_cross_validation(train_df.drop(RedfinModel.TARGET_COLUMN, axis=1), train_df[RedfinModel.TARGET_COLUMN])


Cross-validation scores: [ 0.53481349  0.50444493  0.85976462 -0.31979873  0.31207631]
Average cross-validation score: 0.38


In [109]:
test_df = redfin.fetch_data('for_sale')
# test_df = redfin.filter_data(test_df)
result_df = redfin.predict(test_df)

Fetched properties (805): Rockingham County, New Hampshire for_sale
filtering column: style allowed_values: ['SINGLE_FAMILY', 'TOWNHOUSE']
filtering column: beds allowed_values: [3, 4, 5]
filtering column: full_baths allowed_values: [2, 3]
Filtered data shape: (168, 44) (from (805, 43))
Dropping columns: ['city=Auburn' 'city=Brentwood' 'city=Chester' 'city=Exeter'
 'city=Fremont' 'city=Greenland' 'city=Hampton Falls' 'city=Newfields'
 'city=Newmarket' 'city=North Hampton' 'city=Northwood' 'city=Nottingham'
 'city=Portsmouth' 'city=Raymond' 'city=Rye' 'city=Stratham'
 'nearby_schools=Auburn School District, Pinkerton Academy School District'
 'nearby_schools=Auburn School District, Pinkerton Academy School District, Making Community Connections Charter School, Mill Falls Charter School District'
 'nearby_schools=Auburn School District, Pinkerton Academy School District, Mill Falls Charter School District, Kreiva Academy Public Charter School District, Polaris Charter School, Making Comm

In [110]:
print(redfin.trained_columns)
# remove exponent formatting
result_df = result_df.sort_values(by=['diff_percent'], ascending=False)
result_df[RedfinModel.OUTPUT_COLUMNS].head(25)


['age' 'beds' 'days_on_mls' 'full_baths' 'half_baths' 'hoa_fee' 'lot_sqft'
 'parking_garage' 'sqft' 'stories' 'city=Atkinson' 'city=Candia'
 'city=Danville' 'city=Deerfield' 'city=Derry' 'city=East Kingston'
 'city=Epping' 'city=Hampstead' 'city=Hampton' 'city=Kingston'
 'city=Londonderry' 'city=Plaistow' 'city=Salem' 'city=Sandown'
 'city=Seabrook' 'city=Windham' 'nearby_schools'
 'nearby_schools=Birches Academy for Academic, Hill View Montessori Charter Public (District), Salem School District, Granite State Arts Academy Charter School'
 'nearby_schools=Birches Academy for Academic, Salem School District, Granite State Arts Academy Charter School'
 'nearby_schools=Birches Academy for Academic, Windham School District, Granite State Arts Academy Charter School, Windham Academy Public Charter School'
 'nearby_schools=Candia School District'
 'nearby_schools=Deerfield School District'
 'nearby_schools=Epping School District'
 'nearby_schools=Great Bay Elearning Charter School, Virtual L

,readable_address,style,sqft,beds,full_baths,list_price,predicted,diff,diff_percent,property_url
273,"93 Jenness Rd, Epping, NH",SINGLE_FAMILY,3732,4,2,599000,898343.00,299343.0,49.97379,https://www.realtor.com/realestateandhomes-det...
244,"47 South Rd, Deerfield, NH",SINGLE_FAMILY,4394,4,2,599900,799275.00,199375.0,33.234706,https://www.realtor.com/realestateandhomes-det...
675,"5 Pine Knoll Dr, Atkinson, NH",SINGLE_FAMILY,3809,3,2,689900,912620.00,222720.0,32.28294,https://www.realtor.com/realestateandhomes-det...
163,"25 Newton Junction Rd, Kingston, NH",SINGLE_FAMILY,3810,3,2,675000,890699.00,215699.0,31.955407,https://www.realtor.com/realestateandhomes-det...
248,"5 Raelynn Dr, Derry, NH",SINGLE_FAMILY,3230,3,2,600000,780707.00,180707.0,30.117833,https://www.realtor.com/realestateandhomes-det...
262,"4 Christina Rd, Raymond, NH",SINGLE_FAMILY,3516,3,2,699000,894368.00,195368.0,27.949642,https://www.realtor.com/realestateandhomes-det...
173,"3 Woodland Dr, Londonderry, NH",SINGLE_FAMILY,3304,4,2,649900,828396.00,178496.0,27.465148,https://www.realtor.com/realestateandhomes-det...
769,"51 E Derry Rd, Derry, NH",SINGLE_FAMILY,3800,5,2,714900,909690.00,194790.0,27.247167,https://www.realtor.com/realestateandhomes-det...
97,"150 Range Rd, Windham, NH",SINGLE_FAMILY,3866,5,3,660000,838750.00,178750.0,27.083333,https://www.realtor.com/realestateandhomes-det...
281,"2 Howard St, Salem, NH",SINGLE_FAMILY,3264,3,3,700000,886356.00,186356.0,26.622286,https://www.realtor.com/realestateandhomes-det...


In [111]:
result_df.to_csv('result.csv')

In [112]:

importances = redfin.print_feature_importances()
print(f"===\nFeature Importances ({len(importances)})\n===")
for importance in importances:
    print(importance)



===
Feature Importances (54)
===
('sqft', np.float64(0.5647340785440914))
('age', np.float64(0.09384677119133086))
('parking_garage', np.float64(0.06179005117387207))
('days_on_mls', np.float64(0.054640487085374424))
('half_baths', np.float64(0.05094597194001184))
('lot_sqft', np.float64(0.03667708452878883))
('full_baths', np.float64(0.025053382143609422))
('beds', np.float64(0.02040478022788252))
('city=Windham', np.float64(0.019268071106079643))
('nearby_schools=Next Charter School District, Windham School District, Granite State Arts Academy Charter School, Windham Academy Public Charter School', np.float64(0.017263127303181566))
('nearby_schools=Salem School District, Lawrence Family Development Charter (District), Greater Lawrence Regional Vocational Technical School District, Granite State Arts Academy Charter School, Birches Academy for Academic', np.float64(0.007075357673747079))
('city=Hampstead', np.float64(0.005984085952946934))
('nearby_schools=Timberlane Regional School D

In [113]:
# test_df.head()
print(set(result_df['style']))

{'SINGLE_FAMILY'}


In [114]:
result_df[RedfinModel.OUTPUT_COLUMNS].to_dict(orient='records')[0]

{'readable_address': '93 Jenness Rd, Epping, NH',
 'style': 'SINGLE_FAMILY',
 'sqft': 3732,
 'beds': 4,
 'full_baths': 2,
 'list_price': 599000,
 'predicted': 898343.0,
 'diff': 299343.0,
 'diff_percent': 49.97378964941569,
 'property_url': 'https://www.realtor.com/realestateandhomes-detail/4864565302'}